# Vision and Gaia Conversion

This example illustrates conversion from Vision or Gaia excel export to power-grid-model input data. 
They function in a similar way since both are [Tabular Converters](../converters/tabular_converter.md).
We can then calculate power-flow with it or convert to a different formats like PGM JSON.

## Vision conversion

### 1. Load the Vision data

To export the Vision file in Excel format, please follow the instructions given in the [Vision Manual](https://phasetophase.nl/pdf/VisionEN.pdf).

Define source and destination paths:

In [1]:
source_file = "data/vision/example.xlsx"
destination_file = "data/vision/sym_output.json"

Instantiate the converter, optionally with a source file path.
Then use `load_input_data()` to load the data and convert it to power-grid-model data.
The additional information that is not used in the powerflow calculation but may be useful to link the results to the source data is stored in `extra_info`.

In [2]:
%%capture cap --no-stderr
from power_grid_model_io.converters import VisionExcelConverter

converter = VisionExcelConverter(source_file=source_file)
input_data, extra_info = converter.load_input_data()

Let's investigate the data we have converted, for one of the components: `nodes`

In [3]:
import pandas as pd

# The node data is stored as a numpy structured array in input_data["node"]
display(input_data["node"])

# We can use pandas to display the data in a convenient tabular format
display(pd.DataFrame(input_data["node"]))

# Notice that the node names were not stored in the numpy array, as we don't need them for the calculations
display({i: extra_info[i]["Name"] for i in input_data["node"]["id"]})

array([(0, 400.), (1, 400.), (2, 400.), (3, 400.)],
      dtype={'names': ['id', 'u_rated'], 'formats': ['<i4', '<f8'], 'offsets': [0, 8], 'itemsize': 16, 'aligned': True})

,id,u_rated
0,0,400.0
1,1,400.0
2,2,400.0
3,3,400.0


{0: 'First Node', 1: 'Second Node', 2: 'Third Node', 3: 'Fourth Node'}

### 2. Validate the data
Before we run a power flow calculation, it is wise validate the data. The most basic method is to use `assert_valid_input_data()`, which will raise a `ValueError` when the data is invalid. For more details on data validation, please consult the [validation Example](https://github.com/alliander-opensource/power-grid-model/blob/main/docs/examples/Validation%20Examples.ipynb).

In [4]:
from power_grid_model import CalculationType
from power_grid_model.validation import assert_valid_input_data

assert_valid_input_data(input_data, calculation_type=CalculationType.power_flow, symmetric=True)

### 3. Run the calculation

Run powerflow calculation with the `input_data` and show the results for `nodes`.

In [5]:
from power_grid_model import PowerGridModel

pgm = PowerGridModel(input_data=input_data)
output_data = pgm.calculate_power_flow()

display(pd.DataFrame(output_data["node"]))

,id,energized,u_pu,u,u_angle
0,0,1,0.999991,399.996506,-0.000040
1,1,1,0.985602,394.240830,-0.003033
2,2,1,0.971222,388.488784,-0.006115
3,3,1,0.966073,386.429212,-0.006102


### Cross referencing objects
The converter has generated unique numerical IDs for all the components in the VisionExcel file, in fact for some special components like _Transformer loads_, multiple PGM components have been created, each with their own numerical ID. To find out which component belongs to which id, some helper functions have been defined:

In [6]:
print("PGM object #0:", converter.lookup_id(0))
print("PGM object #4:", converter.lookup_id(4))
print("PGM object #7:", converter.lookup_id(7))
print("PGM object #9:", converter.lookup_id(9))

print(
    "Node with Number=111:",
    converter.get_node_id(number=111)
)

print(
    "Cables with Number=444:",
    converter.get_branch_id(table="Cables", number=444)
)

print(
    "Source with Node.Number=111 and Subnumber=1:",
    converter.get_appliance_id(table="Sources", node_number=111, sub_number=1)
)

print(
    "Loads with Node.Number=444 and Subnumber=1:",
    converter.get_appliance_id(table="Loads", node_number=444, sub_number=1)
)

PGM object #0: {'table': 'Nodes', 'key': {'Number': 111}}
PGM object #4: {'table': 'Cables', 'key': {'Number': 444}}
PGM object #7: {'table': 'Sources', 'key': {'Node.Number': 111, 'Subnumber': 1}}
PGM object #9: {'table': 'Loads', 'key': {'Node.Number': 444, 'Subnumber': 1}}
Node with Number=111: 0
Cables with Number=444: 4
Source with Node.Number=111 and Subnumber=1: 7
Loads with Node.Number=444 and Subnumber=1: 9


### Saving the data as a JSON file
The data can be stored in a json file using the PgmJsonConverter. The file will be saved in the `destination_file` path supplied in the constructor.

In [7]:
from power_grid_model_io.converters import PgmJsonConverter

json_converter = PgmJsonConverter(destination_file=destination_file)
json_converter.save(data=output_data, extra_info=extra_info)

For debugging purposes, let's check the output JSON. Notice that the node names are added to the nodes data.

In [8]:
from pathlib import Path
from IPython.display import display, Markdown

with Path(destination_file).open() as json_file:
    display(Markdown(f"<pre style='max-height: 160px; white-space: pre'>{json_file.read()}</div>"))

<pre style='max-height: 160px; white-space: pre'>{
  "line":
    [
      {"id": 4, "energized": 1, "loading": 0.4494033468128292, "p_from": 92766.12651731551, "q_from": 10913.43660532122, "i_from": 134.82094421436187, "s_from": 93405.87415984583, "p_to": -91463.07558984967, "q_to": -10483.005663479944, "i_to": 134.82100404384875, "s_to": 92061.86835001291, "id_reference": {"table": "Cables", "key": {"Number": 444}}},
      {"id": 5, "energized": 1, "loading": 0.44940353822035317, "p_from": 91463.07558984793, "q_from": 10483.005663480168, "i_from": 134.82100404384624, "s_from": 92061.86835001121, "p_to": -90160.02352913805, "q_to": -10052.564084910864, "i_to": 134.82106146610596, "s_to": 90718.70748338496, "id_reference": {"table": "Cables", "key": {"Number": 555}}},
      {"id": 6, "energized": 1, "loading": 0.15748825414612666, "p_from": 30160.023529136957, "q_from": 10052.564084910147, "i_from": 47.24631623927688, "s_from": 31791.21048276269, "p_to": -29999.99999999949, "q_to": -9999.999999999573, "i_to": 47.246476243837996, "s_to": 31622.776601683174, "id_reference": {"table": "Cables", "key": {"Number": 666}}}
    ],
  "node":
    [
      {"id": 0, "energized": 1, "u_pu": 0.9999912644198532, "u": 399.9965057679413, "u_angle": -3.96611370590559e-05, "id_reference": {"table": "Nodes", "key": {"Number": 111}}, "Name": "First Node"},
      {"id": 1, "energized": 1, "u_pu": 0.9856020738558822, "u": 394.24082954235286, "u_angle": -0.003033170356205173, "id_reference": {"table": "Nodes", "key": {"Number": 222}}, "Name": "Second Node"},
      {"id": 2, "energized": 1, "u_pu": 0.9712219592491987, "u": 388.4887836996795, "u_angle": -0.006115298015992432, "id_reference": {"table": "Nodes", "key": {"Number": 333}}, "Name": "Third Node"},
      {"id": 3, "energized": 1, "u_pu": 0.966073029937675, "u": 386.42921197506996, "u_angle": -0.006102268973727995, "id_reference": {"table": "Nodes", "key": {"Number": 444}}, "Name": "Fourth Node"}
    ],
  "source":
    [
      {"id": 7, "energized": 1, "p": 92766.12651730014, "q": 10913.436605231671, "i": 134.82094421432475, "s": 93405.8741598201, "pf": 0.9931508842642452, "id_reference": {"table": "Sources", "key": {"Node.Number": 111, "Subnumber": 1}}, "Name": "Netvoeding"}
    ],
  "sym_load":
    [
      {"id": 8, "energized": 1, "p": 60000.0, "q": -0.0, "i": 89.16863910839885, "s": 60000.0, "pf": 1.0, "id_reference": {"table": "Loads", "key": {"Node.Number": 333, "Subnumber": 1}}},
      {"id": 9, "energized": 1, "p": 30000.0, "q": 10000.0, "i": 47.246476243838906, "s": 31622.776601683792, "pf": 0.9486832980505139, "id_reference": {"table": "Loads", "key": {"Node.Number": 444, "Subnumber": 1}}}
    ]
}
</div>

### Summary

In [9]:
%%capture cap --no-stderr

from power_grid_model import PowerGridModel, CalculationType
from power_grid_model.validation import assert_valid_input_data
from power_grid_model_io.converters import VisionExcelConverter, PgmJsonConverter

source_file = "data/vision/example.xlsx"
destination_file = "data/vision/sym_output.json"

converter = VisionExcelConverter(source_file=source_file)
input_data, extra_info = converter.load_input_data()
assert_valid_input_data(input_data, calculation_type=CalculationType.power_flow, symmetric=True)
pgm = PowerGridModel(input_data=input_data)
output_data = pgm.calculate_power_flow()
json_converter = PgmJsonConverter(destination_file=destination_file)
json_converter.save(data=output_data, extra_info=extra_info)

## Gaia Conversion

```{note}
Gaia Conversion is under development currently and will not work as intended
```

Please refer to the Gaia manual from the software for instructions on Excel exports.
Gaia Conversion is similar to Vision conversion, so only a brief code example is given:


In [ ]:
from power_grid_model_io.converters.gaia_excel_converter import GaiaExcelConverter

source_file = "data/gaia/example.xlsx"
types_file = "data/gaia/types.xlsx"

gaia_converter = GaiaExcelConverter(source_file=source_file, types_file=types_file)
input_data, extra_info = gaia_converter.load_input_data()
assert_valid_input_data(input_data, calculation_type=CalculationType.power_flow, symmetric=True)
model = PowerGridModel(input_data=input_data)
output_data = model.calculate_power_flow()